## ETL Banxico

In [2]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
from dateutil.relativedelta import relativedelta
import boto3
import json

In [3]:
# 📌 Cargar variables de entorno
load_dotenv()

# 📌 Obtener los tokens desde .env
BANXICO_TOKEN = os.getenv("BANXICO_TOKEN")
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

if not BANXICO_TOKEN:
    raise ValueError("❌ Error: BANXICO_TOKEN no encontrado en .env")

print("✅ Token de Banxico cargado correctamente.")

# 📌 Configurar fechas desde 2020 hasta hoy
initial_date = "2020-01-01"
final_date = datetime.today().strftime("%Y-%m-%d")

# 📌 Configurar cliente de Amazon S3
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)


✅ Token de Banxico cargado correctamente.


In [4]:
def extract_banxico_data(serie_id, initial_date, final_date):
    """
    Extrae datos de la API de Banxico y los devuelve en un DataFrame.
    """
    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos/{initial_date}/{final_date}?token={BANXICO_TOKEN}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None

    data = response.json()
    
    if "bmx" not in data or "series" not in data["bmx"] or not data["bmx"]["series"]:
        print("⚠️ No se encontraron datos en la respuesta de Banxico.")
        return None

    series_data = data["bmx"]["series"][0]["datos"]

    if not series_data:
        print("⚠️ No hay datos disponibles en este rango de fechas.")
        return None

    # 📌 Convertir datos a DataFrame
    df = pd.DataFrame(series_data)
    df.columns = ["timestamp", "value"]

    # 📌 Convertir fechas correctamente
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%d/%m/%Y", dayfirst=True)

    # 📌 Convertir valores a numérico
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

    return df


In [5]:
# 📌 Extraer Tipo de Cambio
df_tipo_cambio = extract_banxico_data("SF43718", initial_date, final_date)
df_tipo_cambio.to_csv("tipo_de_cambio.csv", index=False)

# 📌 Extraer Tasa de Interés
df_tasa_interes = extract_banxico_data("SF282", initial_date, final_date)
df_tasa_interes.to_csv("tasa_de_interes.csv", index=False)

# 📌 Extraer INPC desde Banxico
df_inpc = extract_banxico_data("SP1", initial_date, final_date)
df_inpc.to_csv("inpc.csv", index=False)

print("✅ Datos guardados localmente en CSV.")


✅ Datos guardados localmente en CSV.


In [6]:
# Toma"default"
s3_client = boto3.client("s3")

# Probar conexión
response = s3_client.list_buckets()
for bucket in response["Buckets"]:
    print(f"✅ Bucket encontrado: {bucket['Name']}")
# 📌 Probar conexión
try:
    response = s3_client.list_objects_v2(Bucket="banxico-hw", Prefix="raw/")
    for obj in response.get("Contents", []):
        print(f"📂 Archivo en S3: {obj['Key']} - Tamaño: {obj['Size']} bytes")
except Exception as e:
    print(f"❌ Error al conectar con S3: {e}")


✅ Bucket encontrado: arquitectura-athena-queries-sofia
✅ Bucket encontrado: banxico-hw
✅ Bucket encontrado: itam-analytics-sofia
✅ Bucket encontrado: sofia-temp-flights
📂 Archivo en S3: raw/ - Tamaño: 0 bytes
📂 Archivo en S3: raw/inpc.csv - Tamaño: 1190 bytes
📂 Archivo en S3: raw/tasa_de_interes.csv - Tamaño: 1029 bytes
📂 Archivo en S3: raw/tipo_de_cambio.csv - Tamaño: 24694 bytes


In [7]:
def upload_to_s3(file_name, bucket, folder):
    """
    Sube un archivo local a Amazon S3 en la carpeta especificada.
    """
    object_name = f"{folder}/{file_name}"
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        print(f"✅ Archivo {file_name} subido a S3 en {object_name}")
    except Exception as e:
        print(f"❌ Error al subir {file_name}: {e}")

# 📌 Subir los archivos a S3
upload_to_s3("tipo_de_cambio.csv", "banxico-hw", "raw")
upload_to_s3("tasa_de_interes.csv", "banxico-hw", "raw")
upload_to_s3("inpc.csv", "banxico-hw", "raw")

print("✅ Todos los archivos han sido subidos a S3 en la carpeta 'raw/'.")


✅ Archivo tipo_de_cambio.csv subido a S3 en raw/tipo_de_cambio.csv
✅ Archivo tasa_de_interes.csv subido a S3 en raw/tasa_de_interes.csv
✅ Archivo inpc.csv subido a S3 en raw/inpc.csv
✅ Todos los archivos han sido subidos a S3 en la carpeta 'raw/'.


---

### Transforming and cleaning raw data

In [8]:
# 📌 Usar rutas relativas desde donde está el script
file_paths = {
    "tipo_de_cambio": "tipo_de_cambio.csv",
    "tasa_de_interes": "tasa_de_interes.csv",
    "inpc": "inpc.csv"
}

# 📌 Leer los archivos correctamente
df_tipo_cambio = pd.read_csv(file_paths["tipo_de_cambio"], parse_dates=["timestamp"])
df_tasa_interes = pd.read_csv(file_paths["tasa_de_interes"], parse_dates=["timestamp"])
df_inpc = pd.read_csv(file_paths["inpc"], parse_dates=["timestamp"])

# 📌 Renombrar columnas para mantener coherencia
df_tipo_cambio.rename(columns={"timestamp": "date", "value": "tipo_de_cambio"}, inplace=True)
df_tasa_interes.rename(columns={"timestamp": "date", "value": "tasa_de_interes"}, inplace=True)
df_inpc.rename(columns={"timestamp": "date", "value": "inpc"}, inplace=True)

print("✅ Archivos cargados correctamente")


✅ Archivos cargados correctamente


In [9]:
# print head de los dataframes
print(df_tipo_cambio.head())
print(df_tasa_interes.head())
print(df_inpc.head())

        date  tipo_de_cambio
0 2020-01-02         18.8817
1 2020-01-03         18.8673
2 2020-01-06         18.8270
3 2020-01-07         18.8852
4 2020-01-08         18.7980
        date  tasa_de_interes
0 2020-01-01             7.12
1 2020-02-01             6.96
2 2020-03-01             6.81
3 2020-04-01             6.09
4 2020-05-01             5.47
        date     inpc
0 2020-01-01  106.447
1 2020-02-01  106.889
2 2020-03-01  106.838
3 2020-04-01  105.755
4 2020-05-01  106.162


In [10]:
# 📌 Crear la columna con el INPC rezagado 12 meses
df_inpc["inpc_lag_12"] = df_inpc["inpc"].shift(12)

# 📌 Calcular la inflación anualizada en porcentaje
df_inpc["inflacion"] = 100 * (df_inpc["inpc"] / df_inpc["inpc_lag_12"] - 1)

# 📌 Eliminar filas con valores NaN (primeros 12 meses no tienen inpc_lag_12)
df_inpc = df_inpc.dropna()

print("✅ Inflación anualizada calculada correctamente")

✅ Inflación anualizada calculada correctamente


In [11]:
df_inpc

,date,inpc,inpc_lag_12,inflacion
12,2021-01-01,110.210,106.447,3.535093
13,2021-02-01,110.907,106.889,3.759040
14,2021-03-01,111.824,106.838,4.666879
15,2021-04-01,112.190,105.755,6.084819
16,2021-05-01,112.419,106.162,5.893823
17,2021-06-01,113.018,106.743,5.878606
18,2021-07-01,113.682,107.444,5.805815
19,2021-08-01,113.899,107.867,5.592072
20,2021-09-01,114.601,108.114,6.000148
21,2021-10-01,115.561,108.774,6.239543


In [12]:
# 📌 Convertir las fechas a formato datetime estándar
df_tipo_cambio["date"] = pd.to_datetime(df_tipo_cambio["date"], format="%Y-%m-%d")
df_tasa_interes["date"] = pd.to_datetime(df_tasa_interes["date"], format="%Y-%m-%d")
df_inpc["date"] = pd.to_datetime(df_inpc["date"], format="%Y-%m-%d")

print("✅ Fechas convertidas correctamente")


✅ Fechas convertidas correctamente


In [13]:
# 📌 Convertir las fechas a solo "AAAA-MM" (a nivel mensual)
df_tipo_cambio["date"] = df_tipo_cambio["date"].dt.to_period("M")
df_tasa_interes["date"] = df_tasa_interes["date"].dt.to_period("M")
df_inpc["date"] = df_inpc["date"].dt.to_period("M")

print("✅ Fechas convertidas a nivel mensual")


✅ Fechas convertidas a nivel mensual


In [14]:
# 📌 Para el tipo de cambio, tomamos el promedio mensual
df_tipo_cambio = df_tipo_cambio.groupby("date").agg({"tipo_de_cambio": "mean"}).reset_index()

print("✅ Tipo de cambio agregado a nivel mensual")


✅ Tipo de cambio agregado a nivel mensual


In [15]:
# 📌 Crear un DataFrame con todas las fechas mensuales disponibles
df_base = pd.DataFrame({"date": pd.period_range(start=df_inpc["date"].min(), 
                                                end=df_inpc["date"].max(), 
                                                freq="M")})

print("✅ Fechas base creadas correctamente")


✅ Fechas base creadas correctamente


In [16]:
# 📌 Hacer merge con todas las series asegurando alineación de fechas
df_final = df_base.merge(df_tasa_interes, on="date", how="left") \
                  .merge(df_inpc[["date", "inflacion"]], on="date", how="left") \
                  .merge(df_tipo_cambio, on="date", how="left")

print("✅ Datos combinados correctamente con fechas alineadas")


✅ Datos combinados correctamente con fechas alineadas


In [17]:
# 📌 Verificar si hay valores nulos
print("🔍 Valores nulos antes de dropna():")
print(df_final.isna().sum())

# 📌 Verificar el rango de fechas final
print("📅 Rango de fechas después del merge:")
print(df_final["date"].min(), "➡", df_final["date"].max())


🔍 Valores nulos antes de dropna():
date               0
tasa_de_interes    0
inflacion          0
tipo_de_cambio     0
dtype: int64
📅 Rango de fechas después del merge:
2021-01 ➡ 2025-02


In [18]:
df_final

,date,tasa_de_interes,inflacion,tipo_de_cambio
0,2021-01,4.22,3.535093,19.921475
1,2021-02,4.12,3.759040,20.309732
2,2021-03,4.05,4.666879,20.755450
3,2021-04,4.07,6.084819,20.015285
4,2021-05,4.06,5.893823,19.963129
5,2021-06,4.02,5.878606,20.030068
6,2021-07,4.32,5.805815,19.970109
7,2021-08,4.46,5.592072,20.076136
8,2021-09,4.55,6.000148,20.048690
9,2021-10,4.84,6.239543,20.462557


In [26]:
# 📌 Convertir 'date' a string en formato YYYY-MM-DD antes de guardar
df_final["date"] = df_final["date"].astype(str)  # Convierte Period[M] a string
df_final["date"] = pd.to_datetime(df_final["date"]).dt.strftime('%Y-%m-%d')  # Asegura formato correcto

# 📌 Guardar el archivo transformado localmente
file_name = "datos_transformados.csv"
df_final.to_csv(file_name, index=False)
print(f"✅ Archivo '{file_name}' guardado localmente.")

# 📌 Definir el bucket y carpeta en S3
S3_BUCKET_NAME = "banxico-hw"
S3_FOLDER = "transformed/"

# 📌 Configurar el cliente de S3
s3_client = boto3.client("s3")

# 📌 Intentar crear la carpeta en S3 (S3 no maneja carpetas, pero se puede simular con un objeto vacío)
try:
    s3_client.put_object(Bucket=S3_BUCKET_NAME, Key=f"{S3_FOLDER}")
    print(f"✅ Carpeta '{S3_FOLDER}' creada exitosamente en S3")
except Exception as e:
    print(f"❌ Error al crear la carpeta '{S3_FOLDER}' en S3: {e}")

# 📌 Subir el archivo transformado a S3
s3_path = f"{S3_FOLDER}{file_name}"
try:
    s3_client.upload_file(file_name, S3_BUCKET_NAME, s3_path)
    print(f"✅ Archivo '{file_name}' subido exitosamente a S3 en {s3_path}")
except Exception as e:
    print(f"❌ Error al subir '{file_name}' a S3: {e}")



✅ Archivo 'datos_transformados.csv' guardado localmente.
✅ Carpeta 'transformed/' creada exitosamente en S3
✅ Archivo 'datos_transformados.csv' subido exitosamente a S3 en transformed/datos_transformados.csv


In [27]:
df_final.dtypes

date                object
tasa_de_interes    float64
inflacion          float64
tipo_de_cambio     float64
dtype: object

---

## ETL INEGI -- WIP

In [ ]:
# Cargar las variables de entorno desde .env
load_dotenv()

# Obtener el token
INEGI_TOKEN = os.getenv("INEGI_TOKEN")

# Verificar que se cargó correctamente
if not INEGI_TOKEN:
    raise ValueError("❌ Error: INEGI_TOKEN no encontrado en .env")

print("✅ Token de INEGI cargado correctamente:", INEGI_TOKEN[:10] + "...")


In [ ]:
# ID del INPC Mensual en INEGI
serie_id = "539261"

# Construcción de la URL
url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{serie_id}/es/0700/true/BISE/2.0/{INEGI_TOKEN}?type=json"

# Hacer la solicitud a la API
response = requests.get(url)

# Verificar la respuesta
if response.status_code == 200:
    data = response.json()
    print("📡 Código de respuesta:", response.status_code)
    print("🔍 Respuesta de la API:")
    print(json.dumps(data, indent=4)[:1000])  # Muestra los primeros 1000 caracteres
else:
    print("❌ Error en la solicitud:", response.text)


In [ ]:
import requests
import json
from dotenv import load_dotenv
import os

# Cargar el token desde .env
load_dotenv()
INEGI_TOKEN = os.getenv("INEGI_TOKEN")

# ID del INPC Mensual en INEGI
serie_id = "539261"

# Construcción de la URL
url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{serie_id}/es/0700/true/BISE/2.0/{INEGI_TOKEN}?type=json"

# Hacer la solicitud a la API
response = requests.get(url)

# Verificar la respuesta
if response.status_code == 200:
    data = response.json()
    series_data = data["Series"][0]["OBSERVATIONS"]
    
    print(f"📊 Número de datos devueltos por la API: {len(series_data)}")
    print("📡 Primeros datos:")
    print(json.dumps(series_data[:5], indent=4))  # Muestra solo los primeros 5 valores
else:
    print(f"❌ Error {response.status_code}: {response.text}")
